In [122]:
import pandas as pd
from pathlib import Path
import tempfile
import io
from mrich import print
from datetime import datetime

In [129]:
mdf = pd.read_excel("data/InVivo2/JM_InVivo2_EchoRandomisation_max.xlsx", 
                   usecols="B:G", skiprows=1,
    )
# df = df.set_index(["Timepoint", "Animal number", "Rat"])
indices = ["Timepoint", "Treatment", "Rat", "Animal number"]
mdf = mdf.set_index("Randomised number")
mdf.loc[mdf["Rat"] == "NM", "Rat"] = 79
mdf["Treatment"] = mdf["Rat"].apply(lambda x: "Control" if x%4<2 else "IR")
mdf

,Timepoint,Animal number,Rat,Ear Notch,Acquisition Date,Treatment
Randomised number,,,,,,
7,Baseline,M00681267,64,L,2024-08-31,Control
28,Baseline,M00681268,65,R,2024-08-31,Control
69,Baseline,M00681269,66,LL,2024-08-31,IR
76,Baseline,M00681270,67,NM,2024-09-02,IR
25,Baseline,M00681271,68,L,2024-08-31,Control
...,...,...,...,...,...,...
75,25 wk,M00681278,75,NM,2025-03-28,IR
27,25 wk,M00681279,76,L,2025-03-28,Control
78,25 wk,M00681280,77,R,2025-03-28,Control


In [132]:
data = {}

### PSLAX Bmode

keys = [
    "PSLAX_Bmode",
    "PSLAX_Mmode",
    "PSSAX_Mmode",
]

for key in keys:
    for file in Path("data/InVivo2/blinded").glob(f"202?????_InVivo2_Blinded_*_{key}.csv"):
        
        date, _, _, image_num, _, _ = file.name.split("_")
        
        image_num = int(image_num)
        
        match key:
            case "PSLAX_Bmode":
                search = '"Measurement","Mode","Parameter","Units",'
                n_lines = 11
            case "PSLAX_Mmode" | "PSSAX_Mmode":
                search = '"Measurement","Mode","Parameter","Units","Avg","STD","Instance 1","Instance 2","Instance 3"'
                n_lines = 6
            case _:
                raise ValueError

        cmds = [
            "grep",
            f"-A{n_lines}",
            search,
            file,
        ]

        result = subprocess.run(cmds, stdout=subprocess.PIPE, text=True)
        df = pd.read_csv(io.StringIO(result.stdout), index_col=False)

        d = data.setdefault(image_num, {})
        
        d.update({
            "Randomised number":int(image_num),
            f"{key}: Date": datetime.strptime(date, "%Y%m%d").date(),
        })

        match key:
            case "PSLAX_Bmode":
                for i,row in df.iterrows():
                    param = row["Parameter"]
                    unit = row["Units"]
                    value = row[-1]
                    d[f"{key}: {param} [{unit}]"] = value
            case "PSLAX_Mmode" | "PSSAX_Mmode":
                for i,row in df.iterrows():
                    param = row["Measurement"]
                    unit = row["Units"]
                    d[f"{key}: {param} Avg [{unit}]"] = row["Avg"]
                    d[f"{key}: {param} STD [{unit}]"] = row["STD"]

#### COMBINE

df = pd.DataFrame(data.values())
# df = df.set_index("Randomised number")

#### MAP TO RANDOM NUMBERS

for col in indices:
    df[col] = df["Randomised number"].apply(lambda x: mdf.loc[x,col])

#### SORT

timepoints = ["Baseline", "4 wk", "8 wk", "10 wk", "14 wk", "18 wk", "22 wk", "25 wk"]

df["Timepoint"] = pd.Categorical(df["Timepoint"], categories=timepoints, ordered=True)
    
df = df.sort_values(by=indices)
df = df.set_index(indices + ["Randomised number"])

df

PSLAX_Bmode: Date  \
Timepoint Treatment Rat Animal number Randomised number                     
Baseline  Control   64  M00681267     7                        2025-04-24   
                    65  M00681268     28                       2025-04-26   
                    68  M00681271     25                       2025-04-26   
                    69  M00681272     109                      2025-05-02   
                    72  M00681275     5                        2025-04-24   
...                                                                   ...   
25 wk     IR        71  M00681274     71                       2025-04-28   
                    74  M00681277     88                       2025-04-28   
                    75  M00681278     75                       2025-04-28   
                    78  M00681281     29                       2025-04-26   
                    79  M00681282     124                      2025-05-02   

                                                         PSLAX_Bmode: Heart Rate [BPM]  \
Timepoint Treatment Rat Animal number Randomised number                                  
Baseline  Control   64  M00681267     7                                     423.280423   
                    65  M00681268     28                                    365.408039   
                    68  M00681271     25                                    393.894633   
                    69  M00681272     109                                   394.347683   
                    72  M00681275     5                                     375.175864   
...                                                                                ...   
25 wk     IR        71  M00681274     71                                    410.466906   
                    74  M00681277     88                                    409.137402   
                    75  M00681278     75                                    367.759730   
                    78  M00681281     29                                    436.760692   
                    79  M00681282     124                                   361.391357   

                                                         PSLAX_Bmode: Area [mm2]  \
Timepoint Treatment Rat Animal number Randomised number                            
Baseline  Control   64  M00681267     7                                44.067844   
                    65  M00681268     28                               53.579189   
                    68  M00681271     25                               78.544546   
                    69  M00681272     109                              81.506940   
                    72  M00681275     5                                56.852593   
...                                                                          ...   
25 wk     IR        71  M00681274     71                               31.007125   
                    74  M00681277     88                               29.300926   
                    75  M00681278     75                               74.750803   
                    78  M00681281     29                               54.005644   
                    79  M00681282     124                              40.650569   

                                                         PSLAX_Bmode: Area;s [mm2]  \
Timepoint Treatment Rat Animal number Randomised number                              
Baseline  Control   64  M00681267     7                                  43.706305   
                    65  M00681268     28                                 56.023075   
                    68  M00681271     25                                 52.105674   
                    69  M00681272     109                                46.157460   
                    72  M00681275     5                                  53.216054   
...                                                                            ...   
25 wk     IR        71  M00681274     71                                 31.657742   
                

In [133]:
df.to_excel("InVivo2_processed.xlsx")